# GitHub REST API

Almost the whole of GitHub's functionality can be accessed and controlled using their REST API. [MirageOS](https://mirage.io/), an OCaml library operating system for building unikernels, includes a library for OCaml which wraps many of these APIs, providing an easy and convenient way to query and control GitHub from the command line. Along the way we will learn additional a few more OCaml concepts:

First let's include the required libraries:

In [28]:
#thread;;
#require "github-unix";;
#require "jupyter.notebook";;

Next let us define helper functions:

In [29]:
let show s = ignore (Jupyter_notebook.display "text/html" ("<h3 style='color:red'>"^s^"</h3>"))

let (|>) x f = f x

val show : string -> unit = <fun>


val ( |> ) : 'a -> ('a -> 'b) -> 'b = <fun>


The syntax above is OCaml's way of defining infix functions. Now we can use `|>` in infix position:

In [30]:
let inc x = x + 1

val inc : int -> int = <fun>


In [31]:
5 |> inc |> inc |> inc

- : int = 8


Many of the Github APIs return promises. We will define a helper function `run` that forces the promises:

In [32]:
let run f = f |> Github.Monad.run |> Lwt_main.run

val run : 'a Github.Monad.t -> 'a = <fun>


## Fetching an issue

Let's fetch information about an issue from the OCaml compiler's [github repo](https://github.com/ocaml/ocaml):

In [33]:
let r = Github.Issue.get ~repo:"ocaml" ~user:"ocaml" ~num:8625 () |> run

val r : Github_t.issue Github.Response.t = <obj>


Here we are fetching the issue [#8625](https://github.com/ocaml/ocaml/issues/8625). OCaml has support for labelled arguments. In the function call above, the labels are `repo`, `user` and `num`. Labelled arguments are a way of documenting the interfaces more precisely than just the types `string` or `int`. They can also be passed in any order. The function call above returns the same result as the one above.

In [34]:
let r = Github.Issue.get ~user:"ocaml" ~repo:"ocaml" ~num:8625 () |> run

val r : Github_t.issue Github.Response.t = <obj>


You can look at the type of get by typing just the function name:

In [35]:
Github.Issue.get

- : ?token:Github.Token.t ->
    user:string ->
    repo:string ->
    num:int -> unit -> Github_t.issue Github.Response.t Github.Monad.t
= <fun>


The complete documentation of the Github API is found [here](https://mirage.github.io/ocaml-github/github-unix/Github/index.html). Now we have a value `r` with us which is of type `Github_t.issue Github.Response.t`. We also see that `r` is an object!

In [36]:
r

- : Github_t.issue Github.Response.t = <obj>


OCaml has first-class support for [object oriented programming](https://caml.inria.fr/pub/docs/manual-ocaml/objectexamples.html) in addition to the functional programming and imperative programming features.

We can see by [consulting the documentation](https://mirage.github.io/ocaml-github/github-unix/Github/Response/index.html#type-t) has a method called value which is of type `Github_t.issue` in this case. Object methods are invoked in OCaml as follows:

In [37]:
r#value

- : Github_t.issue =
{Github_t.issue_url = "https://api.github.com/repos/ocaml/ocaml/issues/8625";
 issue_html_url = "https://github.com/ocaml/ocaml/issues/8625";
 issue_number = 8625; issue_state = `Open;
 issue_title = "ocamldep: add a robust mode";
 issue_body =
  "`ocamldep` has a tendency to silently do nothing in case of errors. For instance:\r\n\r\n```\r\n$ ls toto.ml\r\nls: cannot access toto.ml: No such file or directory\r\n$ ocamldep toto.ml\r\n$ echo $?\r\n0\r\n```\r\n\r\nI'm guessing that this behaviour was designed for use in makefiles. However, it tends to dela"... (* string length 513; truncated *);
 issue_user =
  {Github_t.user_login = "diml"; user_id = 1251584L;
   user_url = "https://api.github.com/users/diml";
   user_avatar_url =
    Some "https://avatars1.githubusercontent.com/u/1251584?v=4"};
 issue_labels = []; issue_comments = 3;
 issue_created_at = "2019-04-18T09:25:59Z";
 issue_updated_at = "2019-04-18T15:55:56Z"; issue_closed_at = None;
 issue_milestone = No

## Fetching list of all issues

Let us fetch the list of all the issues from the [OCaml Github API repo](https://github.com/mirage/ocaml-github/issues).

In [38]:
let s = Github.Issue.for_repo ~user:"mirage" ~repo:"ocaml-github" ()

val s : Github_t.issue Github.Stream.t = <abstr>


Unlike getting a single issue, this API call returns a `stream` of issues for the repo. The [Stream API]() has a function called next which returns the optional next value from the stream along with the tail of the stream. We can use the API to print the issue number and issue title as follows:

In [39]:
let rec print_issue_num_title s = 
  match Github.Stream.next s |> run with
  | None -> ()
  | Some ((i : Github_t.issue), s') -> 
      show (Printf.sprintf "%d %s" i.issue_number i.issue_title);
      print_issue_num_title s'

val print_issue_num_title : Github_t.issue Github.Stream.t -> unit = <fun>


In [40]:
print_issue_num_title s

220 Cross-references in code

197 EPIPE when uploading asset

195 Poor error if chmod fails

189 Add webhook pings

185 Add support for GitHub Integrations

...

86 Support Statistics API

84 Rate limit tracking with multiple implicit tokens doesn't work

83 Support check, reset, and fingerprint for tokens

40 using tokens (from cookie jar) to perform git operations (e.g., push)

- : unit = ()


<h3> <span style="color:purple;border-style:solid"> Exercise </span> </h3>

Implement a function to print list of username and URLs of all the comments on the issue [#8625](https://github.com/ocaml/ocaml/issues/8625) that we saw earlier. Use the function [`Github.Issue.comments`](https://mirage.github.io/ocaml-github/github-unix/Github/Issue/index.html#val-comments).

In [44]:
let s' = Github.Issue.comments (failwith "for you to implement")

error: runtime_error

In [45]:
let rec print_issue_comment_user_url s = failwith "for you to implement"

val print_issue_comment_user_url : 'a -> 'b = <fun>


In [46]:
print_issue_num_title s'

error: compile_error

## Github API rate limits

Running anonymously, you may quickly hit GitHub’s rate limiting on REST API calls. There are a couple of ways around this - the best is to take advantage of the git jar command installed. All API calls have a ?token optional parameter which can be either a login token or an OAUTH token. github-unix installs the git jar command which allows you create and store OAUTH tokens in your home directory and retrieve them using the Github_ookie_jar module. Github.Token also contains functions for negotiating GitHub’s 2 factor login.

For example, I can create a new token in the command line by:

```
$ git jar create kayceesrk tutorial
Enter Github password: *******************                                                                                                                                        
Enter Github password: *******************
Enter 2FA code from 'app': 999999
Github cookie jar: created /Users/kc/.github/jar/tutorial
Created token tutorial (9999999999): 9999999999999999999999999999999999999999
```

Copy the token string. Now, you can do:

```
let token = Github.Token.of_string "9999999999999999999999999999999999999999"
let r = Github.Issue.get ~token ~repo:"ocaml" ~user:"ocaml" ~num:8625 () |> run
```